In [1]:
import pandas as pd

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：1


['https://www.gates.com/us/en/search.html?searchType=product&text=Positive+Crankcase+Ventilation+%28PCV%29+Hose']

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

url = input_.loc[a, 'Url']

dict_param = {}
list_param = url.split('?')[1].split('&') if '?' in url else []
for param in list_param:
    dict_param[param.split('=')[0]] = param.split('=')[1]

dict_param['sort'] = 'name-asc'

url = url.split('?')[0] + '?'
for key, value in dict_param.items():
    url += f'{key}={value}&'
    
list_product_number, list_product_name, i = [], [], 1
while True:
    b = 0
    while True:
        b += 1
        print(b, i)
        try:
            resp = requests.get(f'{url}page={i}',
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(10, 10))
        
            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                if html.xpath('//ol[@class="breadcrumbs"]/li') or html.xpath('//span[@class="fontWhite"]/text()') and html.xpath('//span[@class="fontWhite"]/text()')[0] == 'ERROR CODE: 404':
                    break
        except KeyboardInterrupt:
            break
        except:
            continue

    # = = = = = = = = = = = = = = =

    if html.xpath('//span[@class="fontWhite"]/text()') and html.xpath('//span[@class="fontWhite"]/text()')[0] == 'ERROR CODE: 404':
        print('fuck')

    # = = = = = = = = = = = = = = =

    list_div = html.xpath('//div[@class="product-item"]')

    # = = = = = = = = = = = = = = =

    if not list_div:
        print('fuck')

    # = = = = = = = = = = = = = = =

    list_product_number += [div.xpath('./@data-productid')[0].strip() for div in list_div]
    list_product_name += [div.xpath('./div[@class="product__wrapper"]/div[@class="product-description"]/a/h2/text()')[0].strip() for div in list_div]

    # = = = = = = = = = = = = = = =
    
    i += 1

    # = = = = = = = = = = = = = = =

    if not html.xpath('//ol[@class="pagination"][1]/li') or html.xpath('//ol[@class="pagination"][1]/li')[-1].xpath('./@class')[0] == 'pagination-next disabled':
        break

list_product_number, list_product_name

1 1
1 2
1 3
1 4
1 5


(['7347-000000-000575',
  '0094-000000-000002',
  '4980-000000-000000',
  '4217-000000-000002',
  '4219-000000-000013',
  '4219-000000-000000',
  '4219-000000-000001',
  '4177-000000-000004',
  '4177-000000-000001',
  '4261-000000-000000',
  '3353-000000-000000',
  '3353-000000-000001',
  '7744-000000-000000',
  '7250-000000-000079',
  '7250-000000-000059',
  '7250-000000-000061',
  '7250-000000-000062',
  '7250-000000-000063',
  '7250-000000-000069',
  '7250-000000-000065',
  '7250-000000-000064',
  '7250-000000-000068',
  '7250-000000-000067',
  '7250-000000-000066',
  '7250-000000-000070',
  '7745-000000-000000',
  '7250-000000-000028',
  '4663-000000-000001',
  '3333-000000-000000',
  '7796-000000-000000',
  '4278-000000-000000',
  '7250-000000-000072',
  '4219-000000-000004',
  '4219-000000-000002',
  '4219-000000-000003',
  '4219-000000-000008',
  '4219-000000-000007',
  '3129-000000-000002',
  '3150-000000-000000',
  '4688-000000-000000',
  '4110-000000-000011',
  '4110-000000-0

In [6]:
output = pd.DataFrame({'Product No': [i+1 for i in range(len(list_product_name))],
                       'Product Name': list_product_name,
                       'Product Number': list_product_number})

output = output.sort_values(by=['Product No'],
                            ascending=[True],
                            ignore_index=True)

output.to_excel('./test_product.xlsx', index=False)

output

,Product No,Product Name,Product Number
0,1,"1-1/4"" Special One-Piece Couplings - Female JI...",7347-000000-000575
1,2,12 Reel,0094-000000-000002
2,3,12 Reel,4980-000000-000000
3,4,4217IS Standard Diesel Emission Fluid,4217-000000-000002
4,5,4219BD Barricade™ Bio-Diesel Fuel,4219-000000-000013
...,...,...,...
95,96,Field Attachable Type T for G2,7231-000000-000001
96,97,Flange Kits,7253-000000-000004
97,98,Flexible Fuel Fill,4271-000000-000001
98,99,Food Master™ Beverage 150SD CR,3132-000000-000005


In [7]:
crawler_status = 'ok'

crawler_status

'ok'

In [8]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [9]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[False],
                                            ignore_index=True)
    output_error.to_excel('./test_product_error.xlsx', index=False)

output_error

""
